# Laboratorio 4
## 1) Problema minimi quadrati
Data una matrice $A$ di dimensioni $m \cdot n$ con $m \ge n$ ed un vettore $y$ di $m$ componenti, il problema dei minimi quadrati è il seguente: 
$$\alpha^\ast = \arg \min_{\alpha}\|A\alpha − y\|^2_2$$
Questo problema di minimo può essere risolto in due modi:
• **Metodo delle equazioni normali.** Se A ha rango massimo il problema di minimo può essere riscritto in maniera equivalente come segue: $A^T A\alpha = A^T y$.
Questo sistema si può risolvere utilizzando la fattorizzazione LU o di Cholesky (dato che la matrice $A^T A$ è simmetrica).
• **SVD.** Se A non ha rango massimo il problema è sottodeterminato, quindi ha più di una soluzione. In questo caso si considera la decomposizione SVD della matrice $A = USV^T$ dove $U \in \mathbb{R}^{m×m}$ e $V^T \in \mathbb{R}^{n×n}$ matrici ortogonali e $S \in \mathbb{R}^{m×n}$ diagonale. Da questa  ecomposizione si può calcolare esplicitemente la soluzione di minima norma del problema di minimi quadrati come segue: 
$$\alpha = \sum_{i=0}^r \dfrac{(u_i^T \: y) \: v_i}{s_i}.$$

Assegnata una matrice $A$ di numeri casuali di dimensione $m × n$ con $m > n$, generata utilizzando la funzione `np.random.rand`, scegliere un vettore $\alpha$ (per esempio con elementi costanti) come soluzione per creare un problema test e calcolare il termine noto $y = A\alpha$.
Definito quindi il problema di minimi quadrati con la matrice A ed il termine noto $y$ calcolato:
• Calcolare la soluzione del problema risolvendo le equazioni normali mediante la fattorizzazione LU e Cholesky.
• Calcolare la soluzione del problema usando la SVD della matrice A.
• Calcolare l’errore relativo delle soluzioni trovate, rispetto al vettore $\alpha$, soluzione esatta,  tilizzata per generare il problema test.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
from scipy.linalg import lu_factor as LUdec 

m = 100
n = 10

A = ...

alpha_test = ...
y = ...

print('alpha test', alpha_test)

ATA = A.T@A
ATy = A.T@y

lu, piv = LUdec(...)
alpha_LU = scipy.linalg.lu_solve((lu,piv), ...)

print('alpha LU', alpha_LU)

L = scipy.linalg.cholesky(ATA)
x = scipy.linalg.solve_triangular(np.transpose(L), ..., lower=True)
alpha_chol = scipy.linalg.solve_triangular(L, ..., lower=False)

print('alpha chol', alpha_chol)

U, s, Vh = scipy.linalg.svd(A)

print('Shape of U:', U.shape)
print('Shape of s:', s.shape)
print('Shape of V:', Vh.T.shape)

alpha_svd = np.zeros(s.shape)

for i in range(n):
  ui = U[:, i]
  vi = Vh[i, :]

  alpha_svd = alpha_svd + ...

print('alpha SVD', alpha_svd)

## 2) Approssimazione di un set di dati tramite Minimi Quadrati
Sia $\{(x_i , y_i)\}^m_{i=0}$ un set di dati, che devono essere approssimati da un polinomio di grado $n \in \mathbb{N}$ fissato.
$$
p(x) = \alpha_0 + \alpha_1 x + \dots+ \alpha_n x^n
$$
Per calcolare i coefficienti del polinomio si deve risolvere un problema di minimi quadrati in cui la matrice A è definita come segue: 
$$
A = \begin{bmatrix} 1 & x_0 & x^2_0 & \dots & x^n_0\\ 1 & x_1 & x^2_1 & \dots & x^n_1\\ \vdots & \vdots & \vdots & \vdots & \vdots\\ 1 & x_m & x^2_m & \dots & x^n_m \end{bmatrix}
$$
Mentre il termine noto è:
$$
y = \begin{bmatrix} y_0\\ \vdots\\ y_m \end{bmatrix}
$$

Date le seguenti funzioni:
• $f(x) = exp(x/2) \quad x \in [-1,1]$
• $f(x) = \dfrac{1}{1+25x^2} \quad x \in [-1,1]$
• $f(x) = sin(x) + cos(x) \quad x \in [0, 2\pi]$
Si eseguano le seguenti richieste per ciascuna delle funzioni date:
1. Calcolare $m = 10$ coppie di punti $(x_i, f(x_i))$.
2. Per $n$ fissato, calcolare una soluzione del problema di minimi quadrati, descritto sopra, utilizzando un metodo a scelta tra quelli utilizzati nell’esercizio precedente.
3. Per ciascun valore di $n \in \{1, 2, 3, 5, 7\}$, creare una figura con il grafico della funzione esatta $f(x)$ insieme a quello del polinomio di approssimazione $p(x)$, evidenziando gli $m$ punti noti.
4. Per ciascun valore di $n \in \{1, 2, 3, 5, 7\}$, calcolare e stampare il valore del residuo in norma 2 commesso nei punti $x_i$.

In [3]:
case = 2
m = 10
m_plot = 100

# Grado polinomio approssimante
n = 5

if case==0:
    x = np.linspace(-1,1,m)
    y = np.exp(x/2)
elif case==1:
    x = np.linspace(-1,1,m)
    y = 1/(1+25*(x**2))
elif case==2:
    x = np.linspace(0,2*np.pi,m)
    y = np.sin(x)+np.cos(x)


A = ...

for i in range(n+1):
  A[:, i] = ...
  
U, s, Vh = scipy.linalg.svd(A)


alpha_svd = np.zeros(n+1)

for i in range(n+1):
  ui = U[:, i]
  vi = Vh[i, :]

  alpha_svd = alpha_svd + ...

print(alpha_svd)


x_plot = np.linspace(x[0], x[-1], m_plot)
A_plot = np.zeros((m_plot, n+1))

for i in range(n+1):
  A_plot[:, i] = ...

y_interpolation = ...

plt.plot(x, y, 'o')
plt.plot(x_plot, y_interpolation, 'r')
plt.xlabel('x')
plt.ylabel('y')
plt.title(f'Interpolazione polinomiale di grado {n}')
plt.grid()
plt.show()


res = np.linalg.norm(...)
print('Residual: ', res)

TypeError: 'ellipsis' object does not support item assignment